# Build And Manage Your Data Lake With Delta Lake: Demo

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Big data challenge #1: Data lakes can be messy, siloed, and slow

<img src="https://pages.databricks.com/rs/094-YMS-629/images/bi-and-ml-on-all-data.png" alt='Make all your data ready for BI and ML' width=1000/>


<img src="https://pages.databricks.com/rs/094-YMS-629/images/dl1.png" width=800/>

<img src="https://pages.databricks.com/rs/094-YMS-629/images/dl2.png" width=800/>

<img src="https://pages.databricks.com/rs/094-YMS-629/images/dl3.png" width=800/>

<img src="https://pages.databricks.com/rs/094-YMS-629/images/dl4.png" width=800/>

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Architecture

Delta Lake is an open-source storage layer that brings ACID transactions to Apache Spark™ and big data workloads.

<img src="https://delta.io/wp-content/uploads/2019/04/Delta-Lake-marketecture-0423c.png" width=1012/>

## Demo: Building a reliable data pipeline with Delta Lake

For this demo, we will use a public data set of loans from Lending Club. It includes all funded loans from 2012 to 2017. Each loan includes demographic information provided by the applicant, as well as the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. For a full view of the data please view the data dictionary available [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).


![Loan_Data](https://preview.ibb.co/d3tQ4R/Screen_Shot_2018_02_02_at_11_21_51_PM.png)

https://www.kaggle.com/wendykan/lending-club-loan-data

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Overview
<sp>
1. **BRONZE** - Read raw data from Parquet files using Spark, save data in Delta Lake Bronze table
3. **SILVER** - Perform ETL to clean and conform our data, saving the result as a Silver table
4. **GOLD** - Load the Silver table, then narrow it down to fit our specific use case, saving the result as a Gold table
5. Use the Gold table to demonstrate the features of Delta Lake.

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Import raw data and save it into a Delta Lake table (Bronze)
* This will create a lot of small Parquet files emulating the typical small file problem that occurs with streaming or highly transactional data

In [0]:
# Read Parquet files with Spark
df = spark.read.parquet("/databricks-datasets/samples/lending_club/parquet/")

In [0]:
# Reduce the amount of data (to run on DBCE)
(df, _) = df.randomSplit([0.025, 0.975], seed=123)

# Set up managed database to use
spark.sql("CREATE DATABASE IF NOT EXISTS loans")
spark.sql('USE loans')

# Tidy up paths and tables in case notebook has been run before and they already exist
dbutils.fs.rm("/ml/bronze_loan_stats", recurse=True)
dbutils.fs.rm("/ml/silver_loan_stats", recurse=True)
dbutils.fs.rm("/ml/gold_loan_by_state", recurse=True)
spark.sql('DROP TABLE IF EXISTS bronze_loan_stats')
spark.sql("DROP TABLE IF EXISTS silver_loan_stats")
spark.sql('DROP TABLE IF EXISTS gold_loan_stats')

Out[2]: DataFrame[]

<img src="https://databricks.com/wp-content/uploads/2020/02/Simply-Say-Delta.png" width=800/>

In [0]:
%sql drop table if exists bronze_loan_stats;
drop table if exists silver_loan_stats;
drop table if exists gold_loan_stats;

In [0]:
# Configure destination path
DELTALAKE_BRONZE_PATH = "/ml/bronze_loan_stats"

# Write out the table
df.write.format('delta').mode('overwrite').save(DELTALAKE_BRONZE_PATH)

# Register the SQL table in the database
spark.sql(f"CREATE TABLE bronze_loan_stats USING delta LOCATION '{DELTALAKE_BRONZE_PATH}'") 

# Read the table
loan_stats = spark.read.format("delta").load(DELTALAKE_BRONZE_PATH)

display(loan_stats)

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_il_6m,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,issue_d
null,null,1000.0,1000,1000.0,36 months,6.49%,30.65,A,A2,Customers Service Consultant,< 1 year,RENT,40000.0,Source Verified,Current,n,null,null,debt_consolidation,Debt consolidation,441xx,OH,24.09,0.0,Oct-2003,0,29,null,15,0,2184,19.5%,24.0,w,551.81,551.81,521.19,521.19,448.19,73.0,0.0,0.0,0.0,Aug-2017,30.65,Sep-2017,Aug-2017,0,57,1,INDIVIDUAL,null,null,null,0,0,37821,0,11,0,1,14,35637,76,0,1,761,65,11200,0,0,0,2,2521,39,95.1,0.0,0,149.0,105,16,14,0,54,76,null,29,2,1,4,1,2,19,4,5,4,15,0,0,0,0,87.5,100.0,0,0,57980,37821,800,46780,null,null,null,null,null,null,null,null,null,null,null,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Mar-2016
null,null,1000.0,1000,1000.0,36 months,6.49%,30.65,A,A2,dental assistant,8 years,RENT,28000.0,Not Verified,Current,n,null,null,credit_card,Credit card refinancing,142xx,NY,26.49,0.0,Sep-2004,0,null,null,6,0,3651,40.1%,19.0,w,579.33,579.33,490.04,490.04,420.67,69.37,0.0,0.0,0.0,Aug-2017,30.65,Sep-2017,Aug-2017,0,null,1,INDIVIDUAL,null,null,null,0,0,11743,0,3,0,1,13,8092,46,1,1,2697,44,9100,0,0,0,2,1957,4825,40.4,0.0,0,124.0,139,9,9,0,25,null,null,null,0,2,3,2,12,4,3,15,3,6,0,0,0,1,100.0,0.0,0,0,26793,11743,8100,17693,null,null,null,null,null,null,null,null,null,null,null,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Apr-2016
null,null,1000.0,1000,1000.0,36 months,6.99%,30.88,A,A2,Manager,9 years,OWN,100000.0,Not Verified,Fully Paid,n,null,null,vacation,Vacation,376xx,TN,14.69,0.0,Dec-1997,1,null,null,20,0,10215,8%,48.0,w,0.00,0.0,1000.0,1000.0,1000.0,0.0,0.0,0.0,0.0,May-2017,1000.39,null,Jun-2017,0,null,1,INDIVIDUAL,null,null,null,0,0,572605,0,3,1,2,11,49528,69,3,6,9750,30,128300,0,19,1,9,28630,75750,11.4,0.0,0,170.0,233,10,10,6,11,null,1,null,0,1,3,8,10,19,16,23,3,20,0,0,0,5,100.0,0.0,0,0,720292,59743,85500,71992,null,null,null,null,null,null

In [0]:
dbutils.fs.ls('./')

Out[4]: [FileInfo(path='dbfs:/FileStore/', name='FileStore/', size=0),
 FileInfo(path='dbfs:/databricks/', name='databricks/', size=0),
 FileInfo(path='dbfs:/databricks-datasets/', name='databricks-datasets/', size=0),
 FileInfo(path='dbfs:/databricks-results/', name='databricks-results/', size=0),
 FileInfo(path='dbfs:/dbacademy/', name='dbacademy/', size=0),
 FileInfo(path='dbfs:/delta/', name='delta/', size=0),
 FileInfo(path='dbfs:/demo/', name='demo/', size=0),
 FileInfo(path='dbfs:/loan_by_state.parquet/', name='loan_by_state.parquet/', size=0),
 FileInfo(path='dbfs:/local_disk0/', name='local_disk0/', size=0),
 FileInfo(path='dbfs:/ml/', name='ml/', size=0),
 FileInfo(path='dbfs:/mnt/', name='mnt/', size=0),
 FileInfo(path='dbfs:/tmp/', name='tmp/', size=0),
 FileInfo(path='dbfs:/user/', name='user/', size=0)]

In [0]:
dbutils.fs.ls('dbfs:/ml/')

Out[5]: [FileInfo(path='dbfs:/ml/bronze_loan_stats/', name='bronze_loan_stats/', size=0),
 FileInfo(path='dbfs:/ml/loan_by_state_delta/', name='loan_by_state_delta/', size=0)]

In [0]:
from pyspark.sql.functions import *

In [0]:
loan_stats.dtypes

Out[12]: [('id', 'string'),
 ('member_id', 'string'),
 ('loan_amnt', 'float'),
 ('funded_amnt', 'int'),
 ('funded_amnt_inv', 'double'),
 ('term', 'string'),
 ('int_rate', 'string'),
 ('installment', 'double'),
 ('grade', 'string'),
 ('sub_grade', 'string'),
 ('emp_title', 'string'),
 ('emp_length', 'string'),
 ('home_ownership', 'string'),
 ('annual_inc', 'float'),
 ('verification_status', 'string'),
 ('loan_status', 'string'),
 ('pymnt_plan', 'string'),
 ('url', 'string'),
 ('desc', 'string'),
 ('purpose', 'string'),
 ('title', 'string'),
 ('zip_code', 'string'),
 ('addr_state', 'string'),
 ('dti', 'float'),
 ('delinq_2yrs', 'float'),
 ('earliest_cr_line', 'string'),
 ('inq_last_6mths', 'string'),
 ('mths_since_last_delinq', 'int'),
 ('mths_since_last_record', 'string'),
 ('open_acc', 'int'),
 ('pub_rec', 'int'),
 ('revol_bal', 'int'),
 ('revol_util', 'string'),
 ('total_acc', 'float'),
 ('initial_list_status', 'string'),
 ('out_prncp', 'string'),
 ('out_prncp_inv', 'double'),
 ('total_pymnt', 'string'),
 ('total_pymnt_inv', 'double'),
 ('total_rec_prncp', 'double'),
 ('total_rec_int', 'double'),
 ('total_rec_late_fee', 'double'),
 ('recoveries', 'double'),
 ('collection_recovery_fee', 'double'),
 ('last_pymnt_d', 'string'),
 ('last_pymnt_amnt', 'string'),
 ('next_pymnt_d', 'string'),
 ('last_credit_pull_d', 'string'),
 ('collections_12_mths_ex_med', 'string'),
 ('mths_since_last_major_derog', 'string'),
 ('policy_code', 'string'),
 ('application_type', 'string'),
 ('annual_inc_joint', 'string'),
 ('dti_joint', 'double'),
 ('verification_status_joint', 'string'),
 ('acc_now_delinq', 'int'),
 ('tot_coll_amt', 'int'),
 ('tot_cur_bal', 'int'),
 ('open_acc_6m', 'int'),
 ('open_il_6m', 'int'),
 ('open_il_12m', 'int'),
 ('open_il_24m', 'int'),
 ('mths_since_rcnt_il', 'int'),
 ('total_bal_il', 'int'),
 ('il_util', 'int'),
 ('open_rv_12m', 'int'),
 ('open_rv_24m', 'int'),
 ('max_bal_bc', 'int'),
 ('all_util', 'int'),
 ('total_rev_hi_lim', 'int'),
 ('inq_fi', 'int'),
 ('total_cu_tl', 'int'),
 ('inq_last_12m', 'int'),
 ('acc_open_past_24mths', 'int'),
 ('avg_cur_bal', 'int'),
 ('bc_open_to_buy', 'int'),
 ('bc_util', 'double'),
 ('chargeoff_within_12_mths', 'double'),
 ('delinq_amnt', 'int'),
 ('mo_sin_old_il_acct', 'double'),
 ('mo_sin_old_rev_tl_op', 'int'),
 ('mo_sin_rcnt_rev_tl_op', 'int'),
 ('mo_sin_rcnt_tl', 'int'),
 ('mort_acc', 'int'),
 ('mths_since_recent_bc', 'int'),
 ('mths_since_recent_bc_dlq', 'int'),
 ('mths_since_recent_inq', 'int'),
 ('mths_since_recent_revol_delinq', 'int'),
 ('num_accts_ever_120_pd', 'int'),
 ('num_actv_bc_tl', 'int'),
 ('num_actv_rev_tl', 'int'),
 ('num_bc_sats', 'int'),
 ('num_bc_tl', 'int'),
 ('num_il_tl', 'int'),
 ('num_op_rev_tl', 'int'),
 ('num_rev_accts', 'int'),
 ('num_rev_tl_bal_gt_0', 'int'),
 ('num_sats', 'int'),
 ('num_tl_120dpd_2m', 'int'),
 ('num_tl_30dpd', 'int'),
 ('num_tl_90g_dpd_24m', 'int'),
 ('num_tl_op_past_12m', 'int'),
 ('pct_tl_nvr_dlq', 'double'),
 ('percent_bc_gt_75', 'double'),
 ('pub_rec_bankruptcies', 'int'),
 ('tax_liens', 'int'),
 ('tot_hi_cred_lim', 'int'),
 ('total_bal_ex_mort', 'int'),
 ('total_bc_limit', 'int'),
 ('total_il_high_credit_limit', 'int'),
 ('revol_bal_joint', 'int'),
 ('sec_app_earliest_cr_line', 'string'),
 ('sec_app_inq_last_6mths', 'int'),
 ('sec_app_mort_acc', 'int'),
 ('sec_app_open_acc', 'int'),
 ('sec_app_revol_util', 'double'),
 ('sec_app_open_il_6m', 'int'),
 ('sec_app_num_rev_accts', 'int'),
 ('sec_app_chargeoff_within_12_mths', 'int'),
 ('sec_app_collections_12_mths_ex_med', 'int'),
 ('sec_app_mths_since_last_major_derog', 'int'),
 ('hardship_flag', 'string'),
 ('hardship_type', 'string'),
 ('hardship_reason', 'string'),
 ('hardship_status', 'string'),
 ('deferral_term', 'int'),
 ('hardship_amount', 'double'),
 ('hardship_start_date', 'string'),
 ('hardship_end_date', 'string'),
 ('payment_plan_start_date', 'string'),
 ('hardship_length', 'int'),
 ('hardship_dpd', 'int'),
 ('hardship_loan_status', 'string'),
 ('orig_projected_additional_accrued

In [0]:

# Selecting only the columns we are interested in
loan_stats = loan_stats.select("loan_status", "int_rate", "revol_util", "issue_d", "earliest_cr_line", "emp_length", "verification_status", \
                               "total_pymnt", "loan_amnt", "grade", "annual_inc", "dti", "addr_state", "term", "home_ownership", "purpose", \
                               "application_type", "delinq_2yrs", "total_acc")

# Creating 'bad_loan' label, which includes charged off, defaulted, and late repayments on loans
loan_stats = loan_stats.filter(loan_stats.loan_status.isin(["Default", "Charged Off", "Fully Paid"])) \
                       .withColumn("bad_loan", (~(loan_stats.loan_status == "Fully Paid")).cast("string"))

# Transforming string columns into numeric columns
loan_stats = loan_stats.withColumn('int_rate', regexp_replace('int_rate', '%', '').cast('float')) \
                       .withColumn('revol_util', regexp_replace('revol_util', '%', '').cast('float')) \
                       .withColumn('issue_year',  substring(loan_stats.issue_d, 5, 4).cast('double') ) \
                       .withColumn('earliest_year', substring(loan_stats.earliest_cr_line, 5, 4).cast('double'))

# Converting emp_length into numeric column
loan_stats = loan_stats.withColumn('emp_length', trim(regexp_replace(loan_stats.emp_length, "([ ]*+[a-zA-Z].*)|(n/a)", "") ))
loan_stats = loan_stats.withColumn('emp_length', trim(regexp_replace(loan_stats.emp_length, "< 1", "0") ))
loan_stats = loan_stats.withColumn('emp_length', trim(regexp_replace(loan_stats.emp_length, "10\\+", "10") ).cast('float'))

# Bucketing verification_status values together
loan_stats = loan_stats.withColumn('verification_status', trim(regexp_replace(loan_stats.verification_status, 'Source Verified', 'Verified')))

# Calculating the 'credit_length_in_years' column
loan_stats = loan_stats.withColumn('credit_length_in_years', (loan_stats.issue_year - loan_stats.earliest_year))

# Calculating the 'net' column, the total amount of money earned or lost per loan
loan_stats = loan_stats.withColumn('net', round(loan_stats.total_pymnt - loan_stats.loan_amnt, 2))

print('ETL code completed!')

ETL code completed!

In [0]:
display(loan_stats)

loan_status,int_rate,revol_util,issue_d,earliest_cr_line,emp_length,verification_status,total_pymnt,loan_amnt,grade,annual_inc,dti,addr_state,term,home_ownership,purpose,application_type,delinq_2yrs,total_acc,bad_loan,issue_year,earliest_year,credit_length_in_years,net
Fully Paid,6.99,8.0,May-2017,Dec-1997,9.0,Not Verified,1000.0,1000.0,A,100000.0,14.69,TN,36 months,OWN,vacation,INDIVIDUAL,0.0,48.0,false,2017.0,1997.0,20.0,0.0
Fully Paid,8.39,49.9,Apr-2016,Oct-2001,10.0,Not Verified,1027.73,1000.0,B,46500.0,36.98,MD,36 months,MORTGAGE,debt_consolidation,INDIVIDUAL,0.0,29.0,false,2016.0,2001.0,15.0,27.73
Fully Paid,9.17,23.5,Oct-2014,Sep-1994,2.0,Not Verified,1139.400233458,1000.0,B,45000.0,11.01,VA,36 months,RENT,credit_card,INDIVIDUAL,0.0,19.0,false,2014.0,1994.0,20.0,139.4
Fully Paid,9.75,63.7,Mar-2016,Jun-1984,10.0,Not Verified,1003.8,1000.0,B,90000.0,18.99,CA,36 months,MORTGAGE,vacation,INDIVIDUAL,0.0,27.0,false,2016.0,1984.0,32.0,3.8
Fully Paid,9.99,94.0,Nov-2015,Mar-2003,5.0,Not Verified,1001.39,1000.0,B,50000.0,21.05,VA,36 months,MORTGAGE,debt_consolidation,INDIVIDUAL,0.0,10.0,false,2015.0,2003.0,12.0,1.39
Fully Paid,10.99,63.0,Nov-2015,Jul-2003,10.0,Verified,1096.1,1000.0,B,50000.0,17.52,TX,36 months,RENT,other,INDIVIDUAL,0.0,13.0,false,2015.0,2003.0,12.0,96.1
Fully Paid,11.44,41.5,Dec-2016,Aug-2010,3.0,Verified,1010.7363616673,1000.0,B,69000.0,16.21,IN,36 months,MORTGAGE,other,INDIVIDUAL,0.0,13.0,false,2016.0,2010.0,6.0,10.74
Fully Paid,11.44,43.0,Dec-2016,Nov-1987,9.0,Verified,1045.8923144216,1000.0,B,60000.0,18.62,PA,36 months,MORTGAGE,home_improvement,INDIVIDUAL,0.0,25.0,false,2016.0,1987.0,29.0,45.89
Fully Paid,11.48,40.3,Jan-2016,Feb-2001,10.0,Verified,1080.96,1000.0,B,95000.0,17.01,AZ,36 months,RENT,other,INDIVIDUAL,0.0,31.0,false,2016.0,2001.0,15.0,80.96
Fully Paid,11.99,39.4,Dec-2015,Apr-1992,6.0,Verified,1017.34,1000.0,C,26000.0,39.34,FL,36 months,RENT,other,INDIVIDUAL,0.0,22.0,false,2015.0,1992.0,23.0,17.34


In [0]:
help(display) ## https://docs.databricks.com/notebooks/visualizations/index.html

Help on method display in module __main__:

display(input=None, *args, **kwargs) method of __main__.PythonShell instance
 Display plots or data.
 
 Display plot:
 - display() # if matplotlib current figure is set
 - display(matplotlib.figure.Figure)
 - display(ggplot.ggplot)
 
 Display dataset:
 - display(spark.DataFrame)
 - display(list) # if list can be converted to DataFrame, e.g., list of named tuples
 - display(pandas.DataFrame)
 - display(koalas.DataFrame)
 
 Display any other value that has a _repr_html_() method
 
 For Spark 2.0 and 2.1:
 - display(DataFrame, streamName='optional', trigger=optional pyspark.sql.streaming.Trigger,
 checkpointLocation='optional')
 
 For Spark 2.2+:
 - display(DataFrame, streamName='optional', trigger=optional interval like '1 second',
 checkpointLocation='optional')

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Save our cleaned and conformed table as a Silver table in Delta Lake

In [0]:
# Configure destination path
DELTALAKE_SILVER_PATH = "/ml/silver_loan_stats"

# Write out the table to distination (disk)
loan_stats.write.format('delta').mode('overwrite').save(DELTALAKE_SILVER_PATH)

# Register the SQL table in the database
spark.sql("CREATE TABLE if not exists silver_loan_stats USING DELTA LOCATION '" + DELTALAKE_SILVER_PATH + "'")

# Read the table from the (disk)
loan_stats = spark.read.format("delta").load(DELTALAKE_SILVER_PATH)

display(loan_stats)

loan_status,int_rate,revol_util,issue_d,earliest_cr_line,emp_length,verification_status,total_pymnt,loan_amnt,grade,annual_inc,dti,addr_state,term,home_ownership,purpose,application_type,delinq_2yrs,total_acc,bad_loan,issue_year,earliest_year,credit_length_in_years,net
Fully Paid,6.99,8.0,May-2017,Dec-1997,9.0,Not Verified,1000.0,1000.0,A,100000.0,14.69,TN,36 months,OWN,vacation,INDIVIDUAL,0.0,48.0,false,2017.0,1997.0,20.0,0.0
Fully Paid,8.39,49.9,Apr-2016,Oct-2001,10.0,Not Verified,1027.73,1000.0,B,46500.0,36.98,MD,36 months,MORTGAGE,debt_consolidation,INDIVIDUAL,0.0,29.0,false,2016.0,2001.0,15.0,27.73
Fully Paid,9.17,23.5,Oct-2014,Sep-1994,2.0,Not Verified,1139.400233458,1000.0,B,45000.0,11.01,VA,36 months,RENT,credit_card,INDIVIDUAL,0.0,19.0,false,2014.0,1994.0,20.0,139.4
Fully Paid,9.75,63.7,Mar-2016,Jun-1984,10.0,Not Verified,1003.8,1000.0,B,90000.0,18.99,CA,36 months,MORTGAGE,vacation,INDIVIDUAL,0.0,27.0,false,2016.0,1984.0,32.0,3.8
Fully Paid,9.99,94.0,Nov-2015,Mar-2003,5.0,Not Verified,1001.39,1000.0,B,50000.0,21.05,VA,36 months,MORTGAGE,debt_consolidation,INDIVIDUAL,0.0,10.0,false,2015.0,2003.0,12.0,1.39
Fully Paid,10.99,63.0,Nov-2015,Jul-2003,10.0,Verified,1096.1,1000.0,B,50000.0,17.52,TX,36 months,RENT,other,INDIVIDUAL,0.0,13.0,false,2015.0,2003.0,12.0,96.1
Fully Paid,11.44,41.5,Dec-2016,Aug-2010,3.0,Verified,1010.7363616673,1000.0,B,69000.0,16.21,IN,36 months,MORTGAGE,other,INDIVIDUAL,0.0,13.0,false,2016.0,2010.0,6.0,10.74
Fully Paid,11.44,43.0,Dec-2016,Nov-1987,9.0,Verified,1045.8923144216,1000.0,B,60000.0,18.62,PA,36 months,MORTGAGE,home_improvement,INDIVIDUAL,0.0,25.0,false,2016.0,1987.0,29.0,45.89
Fully Paid,11.48,40.3,Jan-2016,Feb-2001,10.0,Verified,1080.96,1000.0,B,95000.0,17.01,AZ,36 months,RENT,other,INDIVIDUAL,0.0,31.0,false,2016.0,2001.0,15.0,80.96
Fully Paid,11.99,39.4,Dec-2015,Apr-1992,6.0,Verified,1017.34,1000.0,C,26000.0,39.34,FL,36 months,RENT,other,INDIVIDUAL,0.0,22.0,false,2015.0,1992.0,23.0,17.34


### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Create Gold table
Now that our Silver table has been cleaned and conformed, and we've evolved the schema, the next step is to create a Gold table. Gold tables are often created to provide clean, reliable data for a specific business unit or use case.

In our case, we'll create a Gold table that includes only 2 columns - `addr_state` and `count` - to provide an aggregated view of our data. For our purposes, this table will allow us to show what Delta Lake can do, but in practice a table like this could be used to feed a downstream reporting or BI tool that needs data formatted in a very specific way. Silver tables often feed multiple downstream Gold tables.

In [0]:
# Aggregate the data
loan_by_state = loan_stats.groupBy("addr_state").count()

# Configure destination path
DELTALAKE_GOLD_PATH = "/ml/gold_loan_by_state"

# Write out the table
loan_by_state.write.format('delta').save(DELTALAKE_GOLD_PATH)

# Register the SQL table in our database
spark.sql(f"CREATE TABLE gold_loan_stats USING delta LOCATION '{DELTALAKE_GOLD_PATH}'")


Out[21]: DataFrame[]

In [0]:
%sql
SELECT *
FROM gold_loan_stats

addr_state,count
AZ,412
SC,197
LA,215
MN,312
NJ,566
DC,53
OR,207
VA,506
RI,71
KY,162


## Stop the notebook before the streaming cell, in case of a "run all"

In [0]:
dbutils.notebook.exit("stop") 

stop

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Unified Batch and Streaming Source and Sink

These cells showcase streaming and batch concurrent queries (inserts and reads)
* This notebook will run an `INSERT` every 10s against our `loan_stats_delta` table
* We will run two streaming queries concurrently against this data
* Note, you can also use `writeStream` but this version is easier to run in DBCE

In [0]:
# Read the insertion of data
loan_by_state_readStream = spark.readStream.format("delta").load(DELTALAKE_GOLD_PATH)
loan_by_state_readStream.createOrReplaceTempView("loan_by_state_readStream")

In [0]:
%sql
SELECT addr_state, sum(`count`) AS loans
FROM loan_by_state_readStream
GROUP BY addr_state

addr_state,loans
AZ,412
SC,197
LA,215
MN,312
NJ,566
DC,53
OR,207
VA,506
RI,71
WY,44


**Wait** until the stream is up and running before executing the code below

In [0]:
import time
i = 1
while i <= 6:
  # Execute Insert statement
  insert_sql = "INSERT INTO gold_loan_stats VALUES ('IA', 450)"
  spark.sql(insert_sql)
  print('gold_loan_stats: inserted new row of data, loop: [%s]' % i)
    
  # Loop through
  i = i + 1
  time.sleep(10)

gold_loan_stats: inserted new row of data, loop: [1]
gold_loan_stats: inserted new row of data, loop: [2]
gold_loan_stats: inserted new row of data, loop: [3]
gold_loan_stats: inserted new row of data, loop: [4]
gold_loan_stats: inserted new row of data, loop: [5]
gold_loan_stats: inserted new row of data, loop: [6]

**Note**: Once the previous cell is finished and the state of Iowa is fully populated in the map (in cell 14), click *Cancel* in Cell 14 to stop the `readStream`.

Let's review our current set of loans using our map visualization.

In [0]:
%sql
-- Review current loans within the `gold_loan_stats` Delta Lake table
SELECT addr_state, SUM(`count`) AS loans
FROM gold_loan_stats
GROUP BY addr_state

addr_state,loans
AZ,412
SC,197
LA,215
MN,312
NJ,566
DC,53
OR,207
VA,506
RI,71
KY,162


Observe that the Iowa (middle state) has the largest number of loans due to the recent stream of data.  Note that the original `gold_loan_stats` table is updated as we're reading `loan_by_state_readStream`.

##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Full DML Support

**Note**: Full DML Support is a feature also in Delta Lake

Delta Lake supports standard DML including UPDATE, DELETE and MERGE INTO providing developers more controls to manage their big datasets.

Let's start by creating a traditional Parquet table

In [0]:
%fs ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks/,databricks/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/dbacademy/,dbacademy/,0
dbfs:/delta/,delta/,0
dbfs:/demo/,demo/,0
dbfs:/loan_by_state.parquet/,loan_by_state.parquet/,0
dbfs:/local_disk0/,local_disk0/,0
dbfs:/ml/,ml/,0


In [0]:
# Load new DataFrame based on current Delta table
lbs_df = sql("SELECT * FROM gold_loan_stats")

# Save DataFrame to Parquet
lbs_df.write.mode("overwrite").parquet("loan_by_state.parquet")

# Reload Parquet Data
lbs_pq = spark.read.parquet("dbfs:/loan_by_state.parquet/") # using absolute path

# Create new table on this parquet data
lbs_pq.createOrReplaceTempView("loan_by_state_pq")

# Review data
display(sql("select * from loan_by_state_pq"))

addr_state,count
AZ,412
SC,197
LA,215
MN,312
NJ,566
DC,53
OR,207
VA,506
RI,71
KY,162


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) DELETE Support

The data was originally supposed to be assigned to `WA` state, so let's `DELETE` those values assigned to `IA`

In [0]:
%sql
-- Attempting to run `DELETE` on the Parquet table
DELETE FROM loan_by_state_pq WHERE addr_state = 'IA'

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Invalid command: 'loan_by_state_pq' is a view not a table.; line 1 pos 0;
'DeleteFromTable ('addr_state = IA)
+- 'UnresolvedV2Relation [loan_by_state_pq], com.databricks.sql.transaction.tahoe.catalog.DeltaCatalog@329e3fec, loans.loan_by_state_pq

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:116)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1$adapted(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:190)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:189)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:189)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:189)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:92)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:180)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:209)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:210)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:206)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:91)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:171)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:836)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:171)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:92)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:89)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:81)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:103)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:836)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:671)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:836)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:666)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:672)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:90)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:36)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:143)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$10(DriverLocal.scala:431)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:239)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:234)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:231)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverL

**Note**: This command fails because the `DELETE` statements are not supported in Parquet, but are supported in Delta Lake.

In [0]:
%sql
-- Running `DELETE` on the Delta Lake table to remove records from Iowa
DELETE FROM gold_loan_stats
WHERE addr_state = 'IA'

As you can see below, after running the `DELETE` command, records containing loans from Iowa have been successfully deleted.

In [0]:
%sql
-- Review current loans within the `gold_loan_stats` Delta Lake table
SELECT addr_state, SUM(`count`) AS loans
FROM gold_loan_stats
GROUP BY addr_state

addr_state,loans
AZ,412
SC,197
LA,215
MN,312
NJ,566
DC,53
OR,207
VA,506
RI,71
KY,162


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) UPDATE Support
The loan data that we assigned to `IA` was originally supposed to be assigned to `WA` state, so let's `UPDATE` those values.

In [0]:
%sql
-- Attempting to run `UPDATE` on the Parquet table
UPDATE loan_by_state_pq SET `count` = 2700 WHERE addr_state = 'WA'

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: UPDATE destination only supports Delta sources.
Some(Relation[addr_state#14928,count#14929L] parquet
);
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.notADeltaSourceException(DeltaErrors.scala:311)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis$$anonfun$apply$1.applyOrElse(DeltaAnalysis.scala:142)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis$$anonfun$apply$1.applyOrElse(DeltaAnalysis.scala:57)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$2(AnalysisHelper.scala:112)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:80)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDown$1(AnalysisHelper.scala:112)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:203)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:110)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:108)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:29)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis.apply(DeltaAnalysis.scala:57)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis.apply(DeltaAnalysis.scala:52)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$2(RuleExecutor.scala:152)
	at scala.collection.LinearSeqOptimized.foldLeft(LinearSeqOptimized.scala:126)
	at scala.collection.LinearSeqOptimized.foldLeft$(LinearSeqOptimized.scala:122)
	at scala.collection.immutable.List.foldLeft(List.scala:89)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1(RuleExecutor.scala:149)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1$adapted(RuleExecutor.scala:141)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:141)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:229)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:223)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:180)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$executeAndTrack$1(RuleExecutor.scala:119)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:88)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:119)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:207)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:210)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:206)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:91)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:171)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:836)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:171)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:92)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:89)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:81)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:103)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:836)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSessio

**Note**: This command fails because the `UPDATE` statements are not supported in Parquet, but are supported in Delta Lake.

In [0]:
%sql
-- Running `UPDATE` on the Delta Lake table
UPDATE gold_loan_stats SET `count` = 1700 WHERE addr_state = 'WA'

As you can see, we successfully ran an `UPDATE` to move those loans from Iowa to Washington state.

In [0]:
%sql
-- Review current loans within the `gold_loan_stats` Delta Lake table
SELECT addr_state, SUM(`count`) AS loans
FROM gold_loan_stats
GROUP BY addr_state

addr_state,loans
AZ,412
SC,197
LA,215
MN,312
NJ,566
DC,53
OR,207
VA,506
RI,71
KY,162


###![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) MERGE INTO Support

#### INSERT or UPDATE with Parquet: 7-step process

With a legacy data pipeline, to insert or update a table, you must:
1. Identify the new rows to be inserted
2. Identify the rows that will be replaced (i.e. updated)
3. Identify all of the rows that are not impacted by the insert or update
4. Create a new temp based on all three insert statements
5. Delete the original table (and all of those associated files)
6. "Rename" the temp table back to the original table name
7. Drop the temp table

![](https://pages.databricks.com/rs/094-YMS-629/images/merge-into-legacy.gif)

#### INSERT or UPDATE with Delta Lake: 2-step process

With Delta Lake, inserting or updating a table is a simple 2-step process: 
1. Identify rows to insert or update
2. Use the `MERGE` command

In [0]:
# Let's create a simple table to merge
items = [('IA', 0), ('CA', 2500), ('OR', 0)]
cols = ['addr_state', 'count']
merge_table = spark.createDataFrame(items, cols)
merge_table.createOrReplaceTempView("merge_table")
display(merge_table)

addr_state,count
IA,0
CA,2500
OR,0


Instead of writing separate `INSERT` and `UPDATE` statements, we can use a `MERGE` statement.

In [0]:
%sql

MERGE INTO gold_loan_stats as d
USING merge_table as m
on d.addr_state = m.addr_state
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *

In [0]:
## Review current loans within the `gold_loan_stats` Delta Lake table
spark.sql("""
          SELECT addr_state, SUM(`count`) AS loans 
          FROM gold_loan_stats 
          GROUP BY addr_state 
          """).show()

+----------+-----+
addr_state|loans|
+----------+-----+
 AZ| 412|
 SC| 197|
 LA| 215|
 MN| 312|
 NJ| 566|
 DC| 53|
 OR| 0|
 VA| 506|
 RI| 71|
 KY| 162|
 WY| 44|
 NH| 76|
 MI| 404|
 NV| 245|
 WI| 214|
 ID| 6|
 CA| 2500|
 CT| 229|
 NE| 21|
 MT| 41|
+----------+-----+
only showing top 20 rows

##![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Schema Evolution
With the `mergeSchema` option, you can evolve your Delta Lake table schema

In [0]:
%sql
select * from gold_loan_stats

addr_state,count
AK,33
AL,214
AR,119
AZ,412
CA,2500
CO,376
CT,229
DC,53
DE,49
FL,1170


In [0]:
# Generate new loans with dollar amounts 
loans = sql("SELECT addr_state, CAST(rand(10)*count AS bigint) AS count, CAST(rand(10) * 10000 * count AS double) AS amount FROM gold_loan_stats")
display(loans)

addr_state,count,amount
AK,5,56413.40555253374
AL,172,1722944.8070131682
AR,68,687335.1436140932
AZ,390,3904131.7223909404
CA,523,5234262.4439425
CO,137,1378574.7704348378
CT,185,1850019.592847161
DC,37,378162.6019729804
DE,35,352570.9527489966
FL,366,3666229.200407528


In [0]:
# Let's write this data out to our Delta table
loans.write.format("delta").mode("append").save(DELTALAKE_GOLD_PATH)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3850734957080676> in <module> 
 1 # Let's write this data out to our Delta table 
 ----> 2 loans . write . format ( "delta" ) . mode ( "append" ) . save ( DELTALAKE_GOLD_PATH ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 825 self . _jwrite . save ( ) 
 826 else : 
 --> 827 self . _jwrite . save ( path ) 
 828 
 829 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : A schema mismatch detected when writing to the Delta table (Table ID: 02e04db2-5fb1-474a-87e4-e9f18d894de7).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- addr_state: string (nullable = true)
-- count: long (nullable = true)


Data schema:
root
-- addr_state: string (nullable = true)
-- count: long (nullable = true)
-- amount: double (nullable = true)

 ;

**Note**: The command above fails because the schema of our new data does not match the schema of our original data.

By adding the **mergeSchema** option, we can successfully migrate our schema, as shown below.

In [0]:
# Add the mergeSchema option
loans.write.option("mergeSchema","true").format("delta").mode("append").save(DELTALAKE_GOLD_PATH)

## No need to read from path/disk again

In [0]:
%sql
select * from gold_loan_stats

addr_state,count,amount
AK,5,56413.40555253374
AL,172,1722944.8070131682
AR,68,687335.1436140932
AZ,390,3904131.7223909404
CA,523,5234262.4439425
CO,137,1378574.7704348378
CT,185,1850019.592847161
DC,37,378162.6019729804
DE,35,352570.9527489966
FL,366,3666229.200407528


In [0]:
%sql
-- Review current loans within the `gold_loan_stats` Delta Lake table
SELECT addr_state, SUM(`amount`) as amount
FROM gold_loan_stats
GROUP BY addr_state
ORDER BY SUM(`amount`) DESC
LIMIT 10

addr_state,amount
CA,5234262.4439425
OH,5185647.865828777
PA,4713668.046846298
NY,4712212.529061128
GA,4232814.448813506
AZ,3904131.7223909404
NJ,3739392.7110618344
FL,3666229.200407528
VA,3505579.6563800476
IL,2892320.3244069736


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Let's Travel back in Time!
Databricks Delta’s time travel capabilities simplify building data pipelines for the following use cases. 

* Audit Data Changes
* Reproduce experiments & reports
* Rollbacks

As you write into a Delta table or directory, every operation is automatically versioned.

You can query by:
1. Using a timestamp
1. Using a version number

using Python, Scala, and/or SQL syntax; for these examples we will use the SQL syntax.  

For more information, refer to [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html)

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Review Delta Lake Table History
All the transactions for this table are stored within this table including the initial set of insertions, update, delete, merge, and inserts with schema modification

In [0]:
%sql
DESCRIBE HISTORY gold_loan_stats

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
10,2020-12-02T00:18:57.000+0000,7342224042861305,peiran@producedelivered.co.nz,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3850734957080627),1201-214927-wot614,9,WriteSerializable,false,"Map(numFiles -> 1, numOutputBytes -> 1767, numOutputRows -> 51)",null
9,2020-12-02T00:10:33.000+0000,7342224042861305,peiran@producedelivered.co.nz,MERGE,"Map(predicate -> (d.`addr_state` = m.`addr_state`), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3850734957080627),1201-214927-wot614,8,WriteSerializable,false,"Map(numTargetRowsCopied -> 48, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetRowsInserted -> 1, numTargetRowsUpdated -> 2, numOutputRows -> 51, numSourceRows -> 3, numTargetFilesRemoved -> 1)",null
8,2020-12-01T23:55:30.000+0000,7342224042861305,peiran@producedelivered.co.nz,UPDATE,Map(predicate -> (addr_state#16890 = WA)),null,List(3850734957080627),1201-214927-wot614,7,WriteSerializable,false,"Map(numRemovedFiles -> 1, numAddedFiles -> 1, numUpdatedRows -> 1, numCopiedRows -> 49)",null
7,2020-12-01T23:43:36.000+0000,7342224042861305,peiran@producedelivered.co.nz,DELETE,"Map(predicate -> [""(`addr_state` = 'IA')""])",null,List(3850734957080627),1201-214927-wot614,6,WriteSerializable,false,"Map(numRemovedFiles -> 6, numDeletedRows -> 6, numAddedFiles -> 1, numCopiedRows -> 0)",null
6,2020-12-01T22:54:37.000+0000,7342224042861305,peiran@producedelivered.co.nz,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3850734957080627),1201-214927-wot614,5,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 702, numOutputRows -> 1)",null
5,2020-12-01T22:54:19.000+0000,7342224042861305,peiran@producedelivered.co.nz,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3850734957080627),1201-214927-wot614,4,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 702, numOutputRows -> 1)",null
4,2020-12-01T22:54:01.000+0000,7342224042861305,peiran@producedelivered.co.nz,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3850734957080627),1201-214927-wot614,3,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 702, numOutputRows -> 1)",null
3,2020-12-01T22:53:42.000+0000,7342224042861305,peiran@producedelivered.co.nz,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3850734957080627),1201-214927-wot614,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 702, numOutputRows -> 1)",null
2,2020-12-01T22:53:21.000+0000,7342224042861305,peiran@producedelivered.co.nz,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3850734957080627),1201-214927-wot614,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 702, numOutputRows -> 1)",null
1,2020-12-01T22:53:02.000+0000,7342224042861305,peiran@producedelivered.co.nz,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3850734957080627),1201-214927-wot614,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputBytes -> 702, numOutputRows -> 1)",null


### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Time Travel via Version Number
Below are SQL syntax examples of Delta Time Travel by using a Version Number

In [0]:
%sql
SELECT * FROM gold_loan_stats VERSION AS OF 8

addr_state,count
AZ,412
SC,197
LA,215
MN,312
NJ,566
DC,53
OR,207
VA,506
RI,71
KY,162


In [0]:
%sql
SELECT * FROM gold_loan_stats VERSION AS OF 0

addr_state,count
AZ,412
SC,197
LA,215
MN,312
NJ,566
DC,53
OR,207
VA,506
RI,71
KY,162


%md ### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Build & Manage Your Data Lake With Delta Lake

To get started with Delta Lake, visit [delta.io](https://delta.io/).

<img src="https://pages.databricks.com/rs/094-YMS-629/images/dl4.png" width=800/>